In [1]:
import json

import pandas as pd
import yaml

In [2]:
# read the technique from params.yaml
with open('../../params.yaml') as f:
    params = yaml.safe_load(f)

technique = params['qa']['system_prompt'].split("/")[0]
technique

'ccot'

In [ ]:
df = pd.read_json('../../data/generated/reports/results.jsonl', lines=True)
print(len(df))
df.head()

300


,id,n_hops,answer,hops,predicted_answer,reference_answers,exact_match,f1,fuzzy_match
0,2hop__129127_90527,2,24 °C (75 °F),[{'question': 'What is the average summer temp...,24 °C (75 °F),[75 ° F],0,0.285714,0
1,2hop__97238_154727,2,1990,[{'question': 'What year did the war Jameel Sa...,1990,[2 August 1990],0,0.500000,0
2,2hop__819850_32467,2,About 150 years old,[{'question': 'How old are some of the private...,About 150 years old,[150 years],0,0.666667,0
3,2hop__71611_90450,2,James K. Polk,[{'question': 'Who was president when the plac...,James K. Polk,"[James K. Polk, President James K. Polk]",1,1.000000,1
4,2hop__612535_47295,2,808,[{'question': 'What is the area code for the s...,808,[808],1,1.000000,1


In [4]:
success_mask = df['exact_match'] == 1
success_df, fail_df = df[success_mask], df[~success_mask]
len(success_df), len(fail_df)

(168, 132)

In [5]:
def format_paragraph(paragraph):
    return f"Paragraph {paragraph['idx']} - {paragraph['paragraph_text']}"

def present_hop(hop):
    print('-'*80)
    print(hop['context'])
    print('-'*80)
    print(hop['question'])
    print('-'*80)
    print(hop['qa_result']['generation'])

def present_row(row):
    print(row['id'])
    print()
    print("Success" if row['f1']>0.99 else "Fail")
    print()
    for hop in row['hops']:
        present_hop(hop)
    print()
    print(f"Reference: {row['reference_answers']}")

In [6]:
i = 0
row = df.iloc[i]
present_row(row)

2hop__129127_90527

Fail

--------------------------------------------------------------------------------
# Climate of Missouri
Summer, June through August, is the hottest time of the year with a mean temperature of 24 ° C (75 ° F) and a mean precipitation of 300 mm (12 inches) with June having more precipitation than either July or August. The extreme highs for the year often occur in July or August. Tropical cyclones and their remains can impact the state during this time of the year, contributing to area rainfall.

# Centerpoint Medical Center
Centerpoint Medical Center is a hospital located in Independence, Missouri at 19600 East 39th Street. It is part of the HCA Midwest Division.
--------------------------------------------------------------------------------
What is the average summer temperature in the state which holds Centerpoint Medical Center?
--------------------------------------------------------------------------------
Thought:
Centerpoint Medical Center is located in 

In [7]:
def make_example(row):
    return {
        'id': row['id'],
        'context': row['hops'][0]['context'],
        'question': row['hops'][0]['question'],
        'generation': row['hops'][0]['qa_result']['generation'],
        'n_hops': row['n_hops'],
    }

In [8]:
filepath = f'{technique}-auto.json'
with open(filepath, 'w') as f:
    json.dump([make_example(row) for i, row in success_df.iterrows()], f, indent=4)